In [44]:
import tweepy
import nltk
import pandas as pd
import pickle
import numpy as np 
import re 
import os
import random
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import style
import time
import statistics
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [45]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
            
        return statistics.mode(votes)

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

In [11]:
# Open dataset 
dataset = pd.read_csv('../resource/training.1600000.processed.noemoticon.csv',header=None, usecols=[0,5], names=['class', 'tweet'], encoding = "ISO-8859-1")
# class = 0 = negative, 4 = positive
df_pos = dataset[:11000]
df_neg = dataset[-11000:]
dt_transformed = pd.concat([df_pos, df_neg])
dt_transformed


,class,tweet
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...


In [12]:
# df_train, df_test = train_test_split(dt_transformed, test_size = 0.40, random_state = 42, stratify=dt_transformed['class'])
# print(f"""
# Training table have {df_train.shape[0]} rows and {df_train.shape[1]} column 
# Test table have {df_test.shape[0]} rows and {df_test.shape[1]} column
# """)
# df_train['class'].value_counts().plot(kind='barh',color='c');df_test['class'].value_counts().plot(kind='barh',color='m')
# dt_transformed.values

In [13]:
all_words = [] 
documents = [] 

def processing_text_to_pos_tag(row_table):
    all_stopwords = stopwords.words('english')
    t_data = re.sub("@[0-9A-za-z:]+", "", row_table[1])  # removing @user
    t_data = re.sub("https?://[A-Za-z0-9./&#]+", "", t_data)  # removing http links
    t_data = t_data.lower()
    t_data = t_data.split()
    ps = PorterStemmer()
    t_data = [ps.stem(word) for word in t_data if not word in set(all_stopwords) if len(word) > 2]
    t_data = ' '.join(t_data)
    
    return row_table[0], t_data

In [14]:
# allowed_word_types = ["J"]

for row in dt_transformed.values:
    c,t = processing_text_to_pos_tag(row)
    documents.append( (t,c) )
    words = word_tokenize(t)
#     pos_t = nltk.pos_tag(words)
    for w in words:
        all_words.append(w)
        
# save_documents = open("../pickled_algos/documents.pickle","wb")
# pickle.dump(documents, save_documents)
# save_documents.close()

all_words = nltk.FreqDist(all_words)
word_features = list(all_words.keys())[:5000]

# save_word_features = open("../pickled_algos/word_features5k.pickle","wb")
# pickle.dump(word_features, save_word_features)
# save_word_features.close()

In [7]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

In [8]:
featuresets = [(find_features(rev), category) for (rev, category) in documents]
random.shuffle(featuresets)

In [17]:
testing_set = featuresets[10000:]
training_set = featuresets[:10000]

In [19]:
# classifier = nltk.NaiveBayesClassifier.train(training_set)
# print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
# # classifier.show_most_informative_features(15)

# save_classifier = open("../pickled_algos/originalnaivebayes5k.pickle","wb")
# pickle.dump(classifier, save_classifier)
# save_classifier.close()

# MNB_classifier = SklearnClassifier(MultinomialNB())
# MNB_classifier.train(training_set)
# print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)

# save_classifier = open("../pickled_algos/MNB_classifier5k.pickle","wb")
# pickle.dump(MNB_classifier, save_classifier)
# save_classifier.close()

# BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
# BernoulliNB_classifier.train(training_set)
# print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)

# save_classifier = open("../pickled_algos/BernoulliNB_classifier5k.pickle","wb")
# pickle.dump(BernoulliNB_classifier, save_classifier)
# save_classifier.close()

# LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
# LogisticRegression_classifier.train(training_set)
# print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

# save_classifier = open("../pickled_algos/LogisticRegression_classifier5k.pickle","wb")
# pickle.dump(LogisticRegression_classifier, save_classifier)
# save_classifier.close()

# LinearSVC_classifier = SklearnClassifier(LinearSVC())
# LinearSVC_classifier.train(training_set)
# print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

# save_classifier = open("../pickled_algos/LinearSVC_classifier5k.pickle","wb")
# pickle.dump(LinearSVC_classifier, save_classifier)
# save_classifier.close()

# SGDC_classifier = SklearnClassifier(SGDClassifier())
# SGDC_classifier.train(training_set)
# print("SGDClassifier accuracy percent:",nltk.classify.accuracy(SGDC_classifier, testing_set)*100)

# save_classifier = open("../pickled_algos/SGDC_classifier5k.pickle","wb")
# pickle.dump(SGDC_classifier, save_classifier)
# save_classifier.close()

In [6]:
documents_f = open("../pickled_algos/documents.pickle", "rb")
documents = pickle.load(documents_f)
documents_f.close()


word_features5k_f = open("../pickled_algos/word_features5k.pickle", "rb")
word_features = pickle.load(word_features5k_f)
word_features5k_f.close()




In [9]:
open_file = open("../pickled_algos/originalnaivebayes5k.pickle", "rb")
classifier = pickle.load(open_file)
open_file.close()


open_file = open("../pickled_algos/MNB_classifier5k.pickle", "rb")
MNB_classifier = pickle.load(open_file)
open_file.close()



open_file = open("../pickled_algos/BernoulliNB_classifier5k.pickle", "rb")
BernoulliNB_classifier = pickle.load(open_file)
open_file.close()


open_file = open("../pickled_algos/LogisticRegression_classifier5k.pickle", "rb")
LogisticRegression_classifier = pickle.load(open_file)
open_file.close()


open_file = open("../pickled_algos/LinearSVC_classifier5k.pickle", "rb")
LinearSVC_classifier = pickle.load(open_file)
open_file.close()


open_file = open("../pickled_algos/SGDC_classifier5k.pickle", "rb")
SGDC_classifier = pickle.load(open_file)
open_file.close()

In [46]:
voted_classifier = VoteClassifier(classifier,
                                  MNB_classifier,
                                  BernoulliNB_classifier,
                                  LogisticRegression_classifier,
                                  SGDC_classifier)

In [64]:
# text_ = 'This might sound bias or racist to some but truth is everyone looks out for their own culture but blacks. Plus they all hate & attack us. For the most part every race dont like blacks are envious but are building & sending their money to their countries'

# def out(text):
#     feats = find_features(text)
#     return voted_classifier.classify(feats),voted_classifier.confidence(feats)

# # c,cc = out(text_)
# # print(c,cc)
# # if cc <= 0.8:
# #     print('Sem certeza')
# # else:
# #     if c == 0:
# #         print('Discurso de odio ou ofensiva')
# #     else:
# #         print('Normal')

In [52]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json


def out(text):
    feats = find_features(text)
    return voted_classifier.classify(feats),voted_classifier.confidence(feats)


ckey="yQuXSscTQvVm2wZ8g3e8tYJrE"
csecret="GVky1dWPwyFz0eOo4JGx4TYG5aB7XkgGcoLBnntAV1KkoZkX9z"
atoken="2555813954-k0L7QV7ds9yuxm5vdUnr0yFhhZkEXX3dZtUKnpc"
asecret="vB3BZplnbCq9UdRbq1NUUJqtsOr2EnzVoiQ6scLw7WNJJ"


class listener(StreamListener):

    def on_data(self, data):
        
        all_data = json.loads(data)
        tweet = all_data["text"]
        sentiment_value, confidence = out(tweet)
        
        print(tweet, sentiment_value, confidence)

        if confidence*100 >= 80:
            output = open("twitter-out.txt","a")
            output.write(str(sentiment_value))
            output.write('\n')
            output.close()

        return True

    def on_error(self, status):
        print(status)

auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, listener())
twitterStream.filter(track=["biden"])

RT @BuckSexton: Last week two of Biden’s top advisors- Fauci and Osterholm- clearly contradicted each other on the  effectiveness of double… 4 1.0
@Christophe_S21 maar alé seg, een sneeuwstorm van 40-50km/hr aan de kust noemen ze dat een stief windje geloof ik,… https://t.co/oUBlBaWtIA 4 1.0
RT @bielconn: 11 dias desde a posse do Biden, e o assessor do Trump posta esse vídeo, onde aparece TRUMP WIN...

mas tudo bem, eu sou o lou… 4 1.0
RT @PressSecKay___: I forgive you for voting for Biden.. Next time you know.
Everyone please go show this veteran some love.
Please go RETW… 4 1.0
Bam! Time to stop playing because the past year/decade has told us that Republicans just don’t care. 4 1.0
This 4 0.8
RT @SteveScalise: RT if you agree → Biden should immediately reverse his decision to allow our American tax dollars to be funneled to group… 0 1.0
RT @syraob: BREAKING: The Biden administration is working to release hurricane disaster aid for Puerto Rico that had been withheld by the T… 4 1.0
R

Third stimulus check update: Who qualifies under Biden, GOP proposals https://t.co/0x0KGHOuNN 4 1.0
RT @AdamSerwer: Republicans were willing to spend trillions and trillions on aid when Trump was president, but now that Biden is president,… 4 1.0
RT @SenSchumer: BREAKING: @SpeakerPelosi and I just filed a concurrent budget resolution to instruct Congressional Committees to craft legi… 4 1.0
#DefundThePolice yeah, sure. Drive around #Chicago and pray you don't get murdered or carjacked. F.Y. #Liberals sta… https://t.co/K95j2AdLas 4 1.0
RT @YDanasmithdutra: MY GOD! Thank you @POTUS 

H/t @CatsChocolates 4 1.0
Biden to meet with GOP lawmakers proposing smaller aid #Digital #digitalhealth #health  https://t.co/JZHI25C0NU 4 1.0
RT @SDSUgrad1983: Excuse me, this makes me sick🤮😡‼️ https://t.co/mG424dqY8S 4 1.0
RT @fakedonald70: Joe Biden will never be Presidential. RT if you agree.! 4 1.0
RT @SteveScalise: RT if you agree → Biden should immediately reverse his decision to allow our American t

RT @SenSchumer: BREAKING: @SpeakerPelosi and I just filed a concurrent budget resolution to instruct Congressional Committees to craft legi… 4 1.0
RT @GeorgePapa19: China is watching Biden’s move in Burma closely. So far, Biden’s incompetence has resulted in the arrest of that country’… 4 1.0
@mccainin4 @Ramila67 @Gechie @CNN @JoeBiden For you to support Joe Biden makes you an idiot He's a criminal communi… https://t.co/B2BFh0KCvV 4 1.0
RT @coachblade: Retired NFL Player Calls for 'Separate Transgender Category' in Sports in Response to Biden's 'Body Blow to the Rights of W… 4 1.0
RT @EamonJavers: Reminder: Here’s how to translate DC spokesperson comments after a meeting like this one:
“A productive meeting” - only mo… 4 1.0
RT @DevilDogIBiteU: 438,035 Americans have died from Covid &amp; Joe Biden still doesn’t have a plan to combat this virus. That’s 2,884 more th… 0 0.6
RT @ChrisFromGA68: Excellent! https://t.co/RMrI30Dkuq 4 1.0
Damn the torpedos! 4 1.0
RT @aedurkinricher: I talked 

RT @ewarren: Canceling student loan debt would immediately put money in the pockets of millions of Americans. It would help dig our economy… 0 1.0
@FoxNews Fox must have teams combing through history looking for any bit of scandals ( in their opinion ) that they… https://t.co/Uq2lWiVcfG 4 0.6
@laescal @tepuescreerque @runewordnews @gaceta_es Le gusta EFE? quizás es un poco mas potable para ud! https://t.co/swNx52EvBC 4 1.0
RT @TheRickyDavila: The Biden Administration confirms that 20 Million doses of the vaccine are missing.

I’m going to guess that criminal s… 4 0.6
RT @evilrwnj: @barronjohn1946 Biden may have hit the nuke button instead of the Diet Coke button ..that’s why It was taken out 😜 4 0.8
RT @AngelTrump2020: “I told you I didn’t work for you,
Now I canceled your pipeline!”
“So you and 7,999 of your union buddies don’t have jo… 0 1.0
RT @SenatorWarnock: Great news: starting Feb. 15, more than 100,000 Georgians are eligible to enroll in health care during a new ACA enroll… 4 1

KeyboardInterrupt: 